In [2]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

23/02/08 09:13:46 WARN Utils: Your hostname, ashish-Lenovo-ideapad-130-15IKB resolves to a loopback address: 127.0.1.1; using 192.168.1.108 instead (on interface wlp2s0)
23/02/08 09:13:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/08 09:13:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/08 09:13:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/02/08 09:13:48 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
# Create an RDD with the Roaming dataset
IRDD = sc.textFile("../in/BroadCastData.csv")

In [4]:
IRDD.collect()

['982120000,United States',
 '982120001,Australia',
 '982120002,Germany',
 '982120003,France',
 '982120004,India',
 '982120005,Canada',
 '982120006,Germany',
 '982120007,France',
 '982120008,India',
 '982120009,Canada',
 '982120010,Germany',
 '982120011,France',
 '982120012,India',
 '982120013,Canada',
 '982120014,United States',
 '982120015,Australia',
 '982120016,Germany',
 '982120017,France',
 '982120018,India',
 '982120019,Canada',
 '982120020,Germany',
 '982120021,France',
 '982120022,India',
 '982120023,Canada',
 '982120024,Germany',
 '982120025,France',
 '982120026,India',
 '982120027,Canada',
 '982120028,United States',
 '982120029,Australia',
 '982120030,Germany',
 '982120031,France',
 '982120032,India',
 '982120033,Canada',
 '982120034,Germany',
 '982120035,France',
 '982120036,India',
 '982120037,Canada',
 '982120038,Germany',
 '982120039,France',
 '982120040,India',
 '982120041,Canada']

In [5]:
#Create a Paired RDD
PairIRDD = IRDD.map(lambda k : (int(k.split(",")[0]), k.split(",")[1]))
#RDD can not be broadcasted. We need to convert it to a value.
#convert the paired RDD to a Map collection and Broadcast
broadcastStates = sc.broadcast(dict(PairIRDD.collect()))

In [6]:
#Create main RDD to be processed
TRDD = sc.textFile("../in/TelecomData.csv")

In [7]:
TRDD.take(5)

['TXCUST00001,982120000,Male,N,PrePaid,Active,Active,InActive,20,N',
 'TXCUST00002,982120001,Male,N,PostPaid,Active,Active,InActive,25,N',
 'TXCUST00003,982120002,Male,N,PrePaid,Active,Active,InActive,20,Y',
 'TXCUST00004,982120003,Male,Y,PrePaid,Active,Active,InActive,25,N',
 'TXCUST00005,982120004,Male,N,PrePaid,Active,Active,InActive,15,N']

In [11]:
TRDD.count()

500

In [8]:
#Access the broadcast value as a look-up data
IRoam = TRDD.map(lambda k : (k.split(",")[0], int(k.split(",")[1]),broadcastStates.value.get(int(k.split(",")[1]))))

In [9]:
IRoam.take(5)

[('TXCUST00001', 982120000, 'United States'),
 ('TXCUST00002', 982120001, 'Australia'),
 ('TXCUST00003', 982120002, 'Germany'),
 ('TXCUST00004', 982120003, 'France'),
 ('TXCUST00005', 982120004, 'India')]

In [27]:
IRoam.take(50)

[('TXCUST00001', 982120000, 'United States'),
 ('TXCUST00002', 982120001, 'Australia'),
 ('TXCUST00003', 982120002, 'Germany'),
 ('TXCUST00004', 982120003, 'France'),
 ('TXCUST00005', 982120004, 'India'),
 ('TXCUST00006', 982120005, 'Canada'),
 ('TXCUST00007', 982120006, 'Germany'),
 ('TXCUST00008', 982120007, 'France'),
 ('TXCUST00009', 982120008, 'India'),
 ('TXCUST00010', 982120009, 'Canada'),
 ('TXCUST00011', 982120010, 'Germany'),
 ('TXCUST00012', 982120011, 'France'),
 ('TXCUST00013', 982120012, 'India'),
 ('TXCUST00014', 982120013, 'Canada'),
 ('TXCUST00015', 982120014, 'United States'),
 ('TXCUST00016', 982120015, 'Australia'),
 ('TXCUST00017', 982120016, 'Germany'),
 ('TXCUST00018', 982120017, 'France'),
 ('TXCUST00019', 982120018, 'India'),
 ('TXCUST00020', 982120019, 'Canada'),
 ('TXCUST00021', 982120020, 'Germany'),
 ('TXCUST00022', 982120021, 'France'),
 ('TXCUST00023', 982120022, 'India'),
 ('TXCUST00024', 982120023, 'Canada'),
 ('TXCUST00025', 982120024, 'Germany'),
 ('T

In [10]:
IRoam.count()

500

## Let us see if what we are doing is effectively a left join operation.

### Checking if we have 982120042 in leftRDD "TRDD"

In [20]:
FilteredRDD = TRDD.filter(lambda record: record.split(",")[1] == "982120042")
FilteredRDD.collect()

['TXCUST00043,982120042,Male,N,PostPaid,Active,Active,InActive,15,N']

### Checking if we have 982120042 in rightRDD "IRDD"

In [28]:
FilteredRDD = IRDD.filter(lambda record: record.split(",")[0] == "982120042")
FilteredRDD.collect()

[]

### Checking if we have 982120042 in the output RDD "IRoam". Note that record in the IRoam is not a string but a tuple.

In [29]:
FilteredRDD = IRoam.filter(lambda record: record[1] == 982120042)
FilteredRDD.collect()

[('TXCUST00043', 982120042, None)]

### Yes, it is effectively doing a left join (but more efficiently as there is less data shuffling through broadcasting).